<a href="https://colab.research.google.com/github/singlethrowdata/Marketing_Automations_Notebooks_With_GPT/blob/main/OpsMOE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Aider
!pip install aider-chat

# Set up Aider with your git repository
import os
os.chdir('/path/to/your/git/repo')
os.environ['OPENAI_API_KEY'] = 'your-openai-api-key'
os.environ['ANTHROPIC_API_KEY'] = 'your-anthropic-api-key'

# Use Aider to edit files
!aider <file1> <file2> ...


FileNotFoundError: [Errno 2] No such file or directory: '/path/to/your/git/repo'

In [ ]:
!pip install google-cloud-bigquery pandas requests

from google.colab import auth
auth.authenticate_user()

import pandas as pd
import requests
import json
from google.cloud import bigquery

# Authenticate with Google Cloud
from google.colab import auth
auth.authenticate_user()

print("Setup complete.")

Setup complete.


In [ ]:
import base64
import requests
import json
from datetime import datetime, timedelta

def get_historical_serps(login, password, keyword, location_name, language_name, date_from, date_to):
    url = "https://api.dataforseo.com/v3/dataforseo_labs/google/historical_serps/live"
    headers = {
        'Authorization': f'Basic {base64.b64encode(f"{login}:{password}".encode()).decode()}'
    }

    # Convert dates to required format (YYYY-MM-DD) and ensure they're strings
    date_from = datetime.strptime(date_from, "%Y-%m-%d").strftime("%Y-%m-%d")
    date_to = datetime.strptime(date_to, "%Y-%m-%d").strftime("%Y-%m-%d")

    # Ensure date_from is not more than 365 days in the past
    max_date_from = (datetime.now() - timedelta(days=365)).strftime("%Y-%m-%d")
    if date_from < max_date_from:
        date_from = max_date_from
        print(f"Warning: date_from adjusted to {max_date_from} (365 days ago)")

    post_data = {
        "keyword": keyword,
        "location_name": location_name,
        "language_name": language_name,
        "date_from": date_from,
        "date_to": date_to
    }

    response = requests.post(url, headers=headers, json=[post_data])
    return response.json()

# Example usage
api_result = get_historical_serps(
    "submissions@singlethrow.com", "6e0e79e416e6c58d",
    "albert einstein", "United States", "English",
    "2023-05-01", "2023-10-01"  # 6-month range for example
)

print("API Response Structure:")
print(json.dumps(api_result, indent=2))

if api_result.get("status_code") == 20000:
    tasks = api_result.get("tasks", [])
    if tasks:
        for task in tasks:
            if task.get("status_code") == 20000:
                result = task.get("result")
                if result:
                    print(f"Retrieved data for {len(result)} months.")
                    for month_data in result:
                        print(f"Data for: {month_data.get('datetime')}")
                        print(f"  Number of items: {month_data.get('items_count')}")
                        print(f"  First organic result: {month_data['items'][0].get('domain') if month_data.get('items') else 'No items'}")
                        print("---")
                else:
                    print("No result data found in the task.")
            else:
                print(f"Task error: {task.get('status_message')}")
    else:
        print("No tasks found in the response.")
else:
    print(f"Error: {api_result.get('status_message')}")

Streaming output truncated to the last 5000 lines.
                    "backlinks": 1253,
                    "time_update": "2023-08-31 03:49:48 +00:00"
                  },
                  "rank_info": {
                    "page_rank": 299,
                    "main_domain_rank": 649
                  }
                },
                {
                  "se_type": "google",
                  "type": "organic",
                  "rank_group": 25,
                  "rank_absolute": 27,
                  "position": "left",
                  "xpath": "/html[1]/body[1]/div[6]/div[1]/div[12]/div[1]/div[2]/div[2]/div[1]/div[1]/div[27]/div[1]",
                  "domain": "history.aip.org",
                  "title": "Albert Einstein in Brief",
                  "url": "https://history.aip.org/exhibits/einstein/inbrief.htm",
                  "breadcrumb": "https://history.aip.org \u203a exhibits \u203a einstein \u203a inbrief",
                  "website_name": "American Institute o

In [ ]:
import json

def inspect_structure(data, prefix=''):
    if isinstance(data, dict):
        for key, value in data.items():
            print(f"{prefix}{key}:")
            inspect_structure(value, prefix + '  ')
    elif isinstance(data, list):
        print(f"{prefix}List with {len(data)} items:")
        if data:  # If the list is not empty
            inspect_structure(data[0], prefix + '  ')
    else:
        print(f"{prefix}{type(data).__name__}: {str(data)[:50]}")  # Print first 50 chars for brevity

# Use this function to inspect your api_result
print("API Result Structure:")
inspect_structure(api_result)

API Result Structure:
version:
  str: 0.1.20240801
status_code:
  int: 20000
status_message:
  str: Ok.
time:
  str: 0.1854 sec.
cost:
  float: 0.0001
tasks_count:
  int: 1
tasks_error:
  int: 0
tasks:
  List with 1 items:
    id:
      str: 08192107-6692-0408-0000-8b8711ece48c
    status_code:
      int: 20000
    status_message:
      str: Ok.
    time:
      str: 0.1289 sec.
    cost:
      float: 0.0001
    result_count:
      int: 1
    path:
      List with 5 items:
        str: v3
    data:
      api:
        str: dataforseo_labs
      function:
        str: historical_serps
      se_type:
        str: google
      keyword:
        str: albert einstein
      location_name:
        str: United States
      language_name:
        str: English
      date_from:
        str: 2023-08-20
      date_to:
        str: 2023-10-01
    result:
      List with 1 items:
        se_type:
          str: google
        keyword:
          str: albert einstein
        location_code:
          int: 

In [ ]:
import pandas as pd

def process_serp_data(api_result):
    all_data = []

    # Navigate to the SERP data
    tasks = api_result.get('tasks', [])
    if not tasks:
        print("No tasks found in the API result.")
        return pd.DataFrame()

    result = tasks[0].get('result', [])
    if not result:
        print("No result found in the task.")
        return pd.DataFrame()

    serp_data = result[0]
    date = serp_data.get('datetime', 'Unknown Date')
    items = serp_data.get('items', [])

    for item in items:
        if item.get('type') == 'organic':
            all_data.append({
                'date': date,
                'domain': item.get('domain'),
                'rank_group': item.get('rank_group'),
                'rank_absolute': item.get('rank_absolute'),
                'position': item.get('position'),
                'title': item.get('title'),
                'url': item.get('url'),
                'description': item.get('description'),
                'etv': item.get('etv'),
                'impressions_etv': item.get('impressions_etv'),
                'estimated_paid_traffic_cost': item.get('estimated_paid_traffic_cost'),
                'previous_rank_absolute': item.get('rank_changes', {}).get('previous_rank_absolute'),
                'is_new': item.get('rank_changes', {}).get('is_new'),
                'is_up': item.get('rank_changes', {}).get('is_up'),
                'is_down': item.get('rank_changes', {}).get('is_down')
            })

    df = pd.DataFrame(all_data)
    print(f"Processed {len(df)} organic results for date: {date}")
    return df

# Use the function
df = process_serp_data(api_result)

# Display the first few rows and basic info about the dataframe
if not df.empty:
    print("\nFirst few rows of the data:")
    print(df.head())
    print("\nDataFrame Info:")
    print(df.info())
else:
    print("No data processed. Please check your api_result.")

# Basic analysis
if not df.empty:
    print("\nTop 10 domains by rank:")
    print(df.groupby('domain')['rank_absolute'].min().sort_values().head(10))

    print("\nDistribution of positions:")
    print(df['position'].value_counts())

    print("\nAverage ETV by rank group:")
    print(df.groupby('rank_group')['etv'].mean().sort_values(ascending=False))

Processed 1 organic results for date: Unknown Date

First few rows of the data:
           date domain rank_group rank_absolute position title   url  \
0  Unknown Date   None       None          None     None  None  None   

  description   etv impressions_etv estimated_paid_traffic_cost  \
0        None  None            None                        None   

  previous_rank_absolute is_new is_up is_down  
0                   None   None  None    None  

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   date                         1 non-null      object
 1   domain                       0 non-null      object
 2   rank_group                   0 non-null      object
 3   rank_absolute                0 non-null      object
 4   position                     0 non-null      object
 5   title                 

In [ ]:
import json

def explore_nested_structure(data, depth=0, max_depth=10):
    if depth > max_depth:
        return "Max depth reached"

    if isinstance(data, dict):
        return {k: explore_nested_structure(v, depth + 1) for k, v in data.items()}
    elif isinstance(data, list):
        if len(data) > 0:
            return [explore_nested_structure(data[0], depth + 1)] + [f"... and {len(data) - 1} more items"]
        else:
            return []
    else:
        return str(type(data))

# Use this function to explore your api_result
print(json.dumps(explore_nested_structure(api_result), indent=2))

{
  "version": "<class 'str'>",
  "status_code": "<class 'int'>",
  "status_message": "<class 'str'>",
  "time": "<class 'str'>",
  "cost": "<class 'float'>",
  "tasks_count": "<class 'int'>",
  "tasks_error": "<class 'int'>",
  "tasks": [
    {
      "id": "<class 'str'>",
      "status_code": "<class 'int'>",
      "status_message": "<class 'str'>",
      "time": "<class 'str'>",
      "cost": "<class 'float'>",
      "result_count": "<class 'int'>",
      "path": [
        "<class 'str'>",
        "... and 4 more items"
      ],
      "data": {
        "api": "<class 'str'>",
        "function": "<class 'str'>",
        "se_type": "<class 'str'>",
        "keyword": "<class 'str'>",
        "location_name": "<class 'str'>",
        "language_name": "<class 'str'>",
        "date_from": "<class 'str'>",
        "date_to": "<class 'str'>"
      },
      "result": [
        {
          "se_type": "<class 'str'>",
          "keyword": "<class 'str'>",
          "location_code": "<class 

In [ ]:
import pandas as pd

def process_serp_data(api_result):
    all_data = []

    # Navigate to the SERP data
    tasks = api_result.get('tasks', [])
    if not tasks:
        print("No tasks found in the API result.")
        return pd.DataFrame()

    task = tasks[0]
    result = task.get('result', [])
    if not result:
        print("No result found in the task.")
        return pd.DataFrame()

    serp_data = result[0]
    date = serp_data.get('datetime', 'Unknown Date')
    items = serp_data.get('items', [])

    if not items:
        print("No items found in the SERP data.")
        return pd.DataFrame()

    organic_items = items[0].get('items', [])

    for item in organic_items:
        if item.get('type') == 'organic':
            all_data.append({
                'date': date,
                'domain': item.get('domain'),
                'rank_group': item.get('rank_group'),
                'rank_absolute': item.get('rank_absolute'),
                'position': item.get('position'),
                'title': item.get('title'),
                'url': item.get('url'),
                'description': item.get('description'),
                'etv': item.get('etv'),
                'impressions_etv': item.get('impressions_etv'),
                'estimated_paid_traffic_cost': item.get('estimated_paid_traffic_cost'),
                'previous_rank_absolute': item.get('rank_changes', {}).get('previous_rank_absolute'),
                'is_new': item.get('rank_changes', {}).get('is_new'),
                'is_up': item.get('rank_changes', {}).get('is_up'),
                'is_down': item.get('rank_changes', {}).get('is_down')
            })

    df = pd.DataFrame(all_data)
    print(f"Processed {len(df)} organic results for date: {date}")
    return df

# Use the function
df = process_serp_data(api_result)

# Display the first few rows and basic info about the dataframe
if not df.empty:
    print("\nFirst few rows of the data:")
    print(df.head())
    print("\nDataFrame Info:")
    print(df.info())

    # Basic analysis
    print("\nTop 10 domains by rank:")
    print(df.groupby('domain')['rank_absolute'].min().sort_values().head(10))

    print("\nDistribution of positions:")
    print(df['position'].value_counts())

    print("\nAverage ETV by rank group:")
    print(df.groupby('rank_group')['etv'].mean().sort_values(ascending=False))
else:
    print("No data processed. Please check your api_result.")

Processed 99 organic results for date: Unknown Date

First few rows of the data:
           date              domain  rank_group  rank_absolute position  \
0  Unknown Date    en.wikipedia.org           1              1     left   
1  Unknown Date  www.nobelprize.org           2              3     left   
2  Unknown Date  www.britannica.com           3              5     left   
3  Unknown Date   www.biography.com           4              6     left   
4  Unknown Date       www.space.com           5              7     left   

                                               title  \
0                                    Albert Einstein   
1    Albert Einstein – Biographical - NobelPrize.org   
2  Albert Einstein | Biography, Education, Discov...   
3              Albert Einstein - Nobel Prize Winners   
4  Albert Einstein: His life, theories and impact...   

                                                 url  \
0      https://en.wikipedia.org/wiki/Albert_Einstein   
1  https://www.nobe

In [ ]:
import pandas as pd

# Assuming the process_serp_data function from earlier is available
# If not, we'll need to define it again

# Create the DataFrame
df = process_serp_data(api_result)

# Display basic information about the DataFrame
print(df.info())

# Display the first few rows
print("\nFirst few rows of the data:")
print(df.head())

# Basic statistics
print("\nBasic statistics:")
print(df.describe())

# Unique values in categorical columns
print("\nUnique values in categorical columns:")
for column in df.select_dtypes(include=['object', 'bool']).columns:
    print(f"{column}: {df[column].nunique()}")

# Check for missing values
print("\nMissing values:")
print(df.isnull().sum())

Processed 99 organic results for date: Unknown Date
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   date                         99 non-null     object 
 1   domain                       99 non-null     object 
 2   rank_group                   99 non-null     int64  
 3   rank_absolute                99 non-null     int64  
 4   position                     99 non-null     object 
 5   title                        99 non-null     object 
 6   url                          99 non-null     object 
 7   description                  99 non-null     object 
 8   etv                          99 non-null     float64
 9   impressions_etv              99 non-null     float64
 10  estimated_paid_traffic_cost  99 non-null     float64
 11  previous_rank_absolute       72 non-null     float64
 12  is_new                      